In [1]:
from typing import List, Optional, cast
import requests
from pystac import Collection, MediaType
from pystac_client import Client, CollectionClient
from datetime import datetime
from dateutil.tz import tzutc

In [2]:
max_description_length = 100

eopf_stac_api_root_endpoint = "https://stac.core.eopf.eodc.eu/"
client = Client.open(url=eopf_stac_api_root_endpoint)
print(
    "Connected to Catalog {id}: {description}".format(
        id=client.id,
        description=client.description
        if len(client.description) <= max_description_length
        else client.description[: max_description_length - 3] + "...",
    )
)

Connected to Catalog eopf-sample-service-stac-api: STAC catalog of the EOPF Sentinel Zarr Samples Service


In [3]:
all_collections: Optional[List[Collection]] = None
# The simplest approach to retrieve all collections may fail due to #18.

try:
    all_collections = [_ for _ in client.get_all_collections()]
    print(
        "* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to be resolved]"
    )
except Exception:
    print(
        "* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to not be resolved]"
    )


* [https://github.com/EOPF-Sample-Service/eopf-stac/issues/18 appears to not be resolved]


In [4]:
client

<Client id=eopf-sample-service-stac-api>

In [9]:
print(all_collections)

[<Collection id=sentinel-2-l2a>, <Collection id=sentinel-3-slstr-l1-rbt>, <Collection id=sentinel-3-olci-l2-lfr>, <Collection id=sentinel-2-l1c>, <Collection id=sentinel-3-slstr-l2-lst>, <Collection id=sentinel-1-l1-slc>, <Collection id=sentinel-3-olci-l1-efr>, <Collection id=sentinel-3-olci-l1-err>, <Collection id=sentinel-1-l2-ocn>, <Collection id=sentinel-1-l1-grd>]


In [ ]:
if all_collections is None:
    # If collection retrieval fails due to #18.
    valid_collections: List[Collection] = []
    for collection_href in [link.absolute_href for link in client.get_child_links()]:
        collection_dict = requests.get(url=collection_href).json()
        try:
            # Attempt to retrieve collections individually.
            valid_collections.append(Collection.from_dict(collection_dict))
        except Exception as e:
            if isinstance(e, TypeError) and "not subscriptable" in str(e).lower():
                # This exception is expected for some collections due to #18.
                continue
            else:
                raise e
    all_collections = valid_collections


In [ ]:
#After troubleshooting for errors, we can see that the available collections where we can retrieve zarr assets are:

#Toprint(all_collections)

[<Collection id=sentinel-2-l2a>, <Collection id=sentinel-3-slstr-l1-rbt>, <Collection id=sentinel-3-olci-l2-lfr>, <Collection id=sentinel-2-l1c>, <Collection id=sentinel-3-slstr-l2-lst>, <Collection id=sentinel-1-l1-slc>, <Collection id=sentinel-3-olci-l1-efr>, <Collection id=sentinel-3-olci-l1-err>, <Collection id=sentinel-1-l2-ocn>, <Collection id=sentinel-1-l1-grd>]


In [11]:
# Here we can get an overview of the available collections inside the STAC catalogue and a descriptio to know
# which collection we are accessing 
# Iterate over all available (valid) collections and report basic information.
for collection in all_collections:
    collection_parent = collection.get_parent()
    print("Collection {id}".format(id=collection.id))
    # if collection_parent is not None:
    #     print(
    #         " - Child of {parent_id}".format(
    #             parent_id=collection_parent.id,
    #         )
    #     )
    # Do not print the entire description as it may be very long.
    print(
        " - Description: {description}".format(
            description=collection.description
            if len(collection.description) <= max_description_length
            else collection.description[: max_description_length - 3] + "..."
        )
    )

Collection sentinel-2-l2a
 - Description: The Sentinel-2 Level-2A Collection 1 product provides orthorectified Surface Reflectance (Bottom-...
Collection sentinel-3-slstr-l1-rbt
 - Description: The Sentinel-3 SLSTR Level-1B RBT product provides radiances and brightness temperatures for each...
Collection sentinel-3-olci-l2-lfr
 - Description: The Sentinel-3 OLCI L2 LFR product provides land and atmospheric geophysical parameters computed ...
Collection sentinel-2-l1c
 - Description: The Sentinel-2 Level-1C product is composed of 110x110 km2 tiles (ortho-images in UTM/WGS84 proje...
Collection sentinel-3-slstr-l2-lst
 - Description: The Sentinel-3 SLSTR Level-2 LST product provides land surface temperature.
Collection sentinel-1-l1-slc
 - Description: The Sentinel-1 Level-1 Single Look Complex (SLC) products consist of focused SAR data, geo-refere...
Collection sentinel-3-olci-l1-efr
 - Description: The Sentinel-3 OLCI L1 EFR product provides TOA radiances at full resolution for each pi

In [ ]:
# To explore data availability, we can define an area of interest, by defining a bounding box composed by the top left corner
# and the low right corner.
# with the .search() argument, we are able to define a series of parameters that allow us filtering the available data that
# matches the criteria

In [13]:
#Lets focus in a spefici area inn Europe... Innsbruck
bbox_search = client.search(
    bbox=(
        11.124756,
        47.311058,
        11.459839,
        47.463624
    )
)

In [14]:
bbox_search

In [ ]:
#The accessed bounding box delivers a pystac_client.item_search.ItemSearch
# To allows us access the infortmation inside, we extract it in a list, which will allow us automatise further or
# more complicated retrievals

In [16]:
contained_items =[]
for item in bbox_search.items():
    contained_items.append(item.collection_id)
    # print(
    #     "bbox search result item ID: {id}, BBOX: [{bbox}]".format(
    #         id=item.id, bbox=item.bbox
    #     )
    # )

In [17]:
print(set(contained_items))
print(len(contained_items))

{'sentinel-3-slstr-l2-lst', 'sentinel-3-olci-l2-lfr', 'sentinel-3-slstr-l1-rbt', 'sentinel-2-l1c', 'sentinel-1-l1-grd', 'sentinel-2-l2a', 'sentinel-3-olci-l1-efr', 'sentinel-3-olci-l1-err', 'sentinel-1-l1-slc'}
124


In [ ]:
#We are able to see, that out of the 11 collections, for the area there are 9 available collections with 124 zarr items
#contained

#It is also possible to filter an interval of interest (a feature not that straightforward in the browser (?))

In [18]:
#As this retrieval can be time consuming, lets revise for the available assets among a specific period of time:
time_frame = client.search(
    datetime="2020-05-01T00:00:00Z/2023-05-31T23:59:59.999999Z")


In [19]:
time_items =[]
for item in time_frame.items():
    time_items.append(item.collection_id)
    # print(
    #     "bbox search result item ID: {id}, BBOX: [{bbox}]".format(
    #         id=item.id, bbox=item.bbox
    #     )
    # )

In [20]:
print(set(time_items))
print(len(time_items))

{'sentinel-2-l2a', 'sentinel-1-l1-grd'}
270


In [21]:
#To start exploring the data inside, we can choose an arbitrary collection:
#For the available items:
selected_collection = client.get_child("sentinel-2-l2a")

In [ ]:
#This cell will list all the items depending on the collection, but it will take a while...
# a_items = list(selected_collection.get_items(recursive=True)) 

In [22]:
#Retrieve the temporal coverage of a specific collection:
start_date = selected_collection.extent.temporal.intervals[0][0]
end_date = selected_collection.extent.temporal.intervals[0][1]

In [23]:
print("Temporal Extent: {start_date} to {end_date}".format(start_date=start_date,end_date=end_date))

Temporal Extent: 2018-06-01 10:20:21.024000+00:00 to 2025-06-01 14:28:01.024000+00:00
